Does Context help?

In [3]:
from pathlib import Path
import pandas as pd

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

/home/ada/humor/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.90it/s]


In [4]:
transcript = Path("/home/ada/humor/data/stand_up_dataset/gillis.txt").read_text()
instruction = "Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. Only output a list of quotes:"
context = "Shane Gillis' debut stand-up comedy special, recorded live at The Creek and The Cave in Austin, TX, offers a blend of edgy, controversial humor and personal anecdotes. Gillis touches on a variety of topics, from his own haircut mishaps to observations on family dynamics, politics, and societal issues. He jokes about his father’s Fox News obsession, racial dynamics in football, and the absurdity of modern social media culture. The special includes candid reflections on his family, like his sister’s struggles with addiction and recovery, and his own experience coaching in the Special Olympics. Gillis also delves into political satire, poking fun at both sides of the political spectrum, and provides irreverent takes on pop culture and current events. His approach mixes dark humor with a candid examination of the absurdities of everyday life."

In [16]:
quotes = [{"comedian": "shane_gillis", "sentence": "Which sucks for her because if she played guitar, everybody would be like, “She’s a genius.”"},
          {"comedian": "shane_gillis", "sentence": "Instead, she’s a hairdresser, so it’s not as cool."},
          {"comedian": "shane_gillis", "sentence": "Actually, based on this crowd, I’m sure it’s actually a lot of your guys’ families too."},
          {"comedian": "shane_gillis", "sentence": "I don’t know if you know these guys, they’re not just f*cking stopping by."},
          {"comedian": "shane_gillis", "sentence": "They’re scurrying around, dude, doing quests."},
          {"comedian": "shane_gillis", "sentence": "If you do heroin long enough, you run out of money. You got to start doing quests."},
          {"comedian": "shane_gillis", "sentence": "It’s like, “You need to gather coppers. Gonna find a scrap yard.”"},
          {"comedian": "shane_gillis", "sentence": "Towards the end, the quest, you need to get titty behind the Home Depot."},
          {"comedian": "shane_gillis", "sentence": "No, if you take heroin, every bus is a bullet train. It’s a five-minute ride, tops."},
          {"comedian": "shane_gillis", "sentence": "Like, an intervention’s supposed to be the worst day of your life, not like you’re tired from easily the best day humanly possible."},
          {"comedian": "shane_gillis", "sentence": "I mean, she took heroin and rode roller coasters. "},
          {"comedian": "shane_gillis", "sentence": "She’s like, “I’m going back tomorrow.”"}]

truth = pd.DataFrame(quotes)

# No Context

In [6]:
prompt = f"\n{instruction}\n'''{transcript}'''"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(input_ids=input_ids, max_new_tokens=180)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = generated_text.replace(prompt, "").strip()

NameError: name 'tokenizer' is not defined

In [7]:
no_context = [{"comedian": "shane_gillis", "sentence": "Which sucks for her because if she played guitar, everybody would be like, “She’s a genius.” Instead, she’s a hairdresser, so it’s not as cool."},
           {"comedian": "shane_gillis", "sentence": "They’re the busiest people. They get a bad rap for being lazy because they take a lot of naps. But if they’re awake, they’re f*cking…very busy. They’re scurrying around, dude, doing quests."},
           {"comedian": "shane_gillis", "sentence": "You know, it’s… You know, it’s like a f*cked up game of Zelda every day. You just wake up to a new quest."},
           {"comedian": "shane_gillis", "sentence": "It’s like, “You need to gather coppers. Gonna find a scrap yard.” It sounds nice. I wish I got quests."}]
no_context_df = pd.DataFrame(no_context)

# Context

In [41]:
prompt = f"{context}\n{instruction}\n'''{transcript}'''"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(input_ids=input_ids, max_new_tokens=180)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = generated_text.replace(prompt, "").strip()

In [9]:
sg_context = [{"comedian": "shane_gillis", "sentence": "She’s a hairdresser, so it’s not as cool."},
           {"comedian": "shane_gillis", "sentence": "They’re the busiest people. They get a bad rap for being lazy because they take a lot of naps. But if they’re awake, they’re f*cking…very busy."},
           {"comedian": "shane_gillis", "sentence": "It’s like a f*cked up game of Zelda every day. You just wake up to a new quest."},
           {"comedian": "shane_gillist", "sentence": "The quest, you need to get titty behind the Home Depot"},
           {"comedian": "shane_gillis", "sentence": "Every bus is a bullet train. It’s a five-minute ride, tops."},
           {"comedian": "shane_gillis", "sentence": "I mean, she took heroin and rode roller coasters. It was… Dude, it was impossible."}]
sg_context_df = pd.DataFrame(sg_context)

Metric

In [10]:
import sys
sys.path.append("..")
from humor.bipartite_metric import bipartite_metric

In [11]:
no_context = bipartite_metric(no_context_df, truth)
print("No Context Model:", no_context.select_dtypes(include='number').mean())

No Context Model: score    20.083333
dtype: float64


In [13]:
context = bipartite_metric(sg_context_df, truth)
print("No Context Model:", context.select_dtypes(include='number').mean())

No Context Model: score    27.75
dtype: float64


# Others

In [40]:
transcripts = pd.read_csv('/home/ada/humor/data/stand_up_dataset/standup_transcripts.csv')
context = pd.read_csv('/home/ada/humor/data/stand_up_dataset/context - gemma2 - gemma2.csv')
ground_truth = pd.read_csv('/home/ada/humor/data/stand_up_dataset/standup_data.csv')
gemma2 = pd.read_csv("/home/ada/humor/data/stand_up_dataset/gemma2 - gemma2.csv")
ground = ground_truth.loc[ground_truth["comedian"].isin(["Anthony_Jeselnik", "Anthony_Jeselnik_2", "Anthony_Jeselnik_3", "Ali_Wong", "Ali_Wong_2", "Ali_Wong_3", "Hasan_Minhaj_3", "Hasan_Minhaj", "Hasan_Minhaj_2", "Joe_List", "Joe_List_2", "Joe_List_3"])]
transcripts = transcripts.loc[transcripts["comedian"].isin(["Anthony_Jeselnik", "Anthony_Jeselnik_2", "Anthony_Jeselnik_3", "Ali_Wong", "Ali_Wong_2", "Ali_Wong_3", "Hasan_Minhaj_3", "Hasan_Minhaj", "Hasan_Minhaj_2", "Joe_List", "Joe_List_2", "Joe_List_3"])]
gemma2 = gemma2.loc[gemma2["comedian"].isin(["Anthony_Jeselnik", "Anthony_Jeselnik_2", "Anthony_Jeselnik_3", "Ali_Wong", "Ali_Wong_2", "Ali_Wong_3", "Hasan_Minhaj_3", "Hasan_Minhaj", "Hasan_Minhaj_2", "Joe_List", "Joe_List_2", "Joe_List_3"])]
merged_df = transcripts.merge(context, on='comedian', how='left')

In [49]:
ground = ground.drop(columns=['laugh_start', 'laugh_end'])

In [51]:
gt = pd.concat([ground, truth], ignore_index=True)

In [86]:
results_dict = {}
for index, row in merged_df.iterrows():
    comedian = row['comedian']
    transcript = row['transcript'] 
    context = row["context"]
    
    instruction = f"Extract the key humorous lines and punchlines for this stand-up comedy transcript. {transcript} Focus on the quotes highlighting the main comedic moments. Output a list of quotes:"
    prompt = f"{context}\n{instruction}"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids=input_ids, max_new_tokens=120)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = generated_text.replace(prompt, "").strip()
    results_dict[comedian] = response
    

In [54]:
context = [
            {"comedian": "shane_gillis", "sentence": "She’s a hairdresser, so it’s not as cool."},
            {"comedian": "shane_gillis", "sentence": "They’re the busiest people. They get a bad rap for being lazy because they take a lot of naps. But if they’re awake, they’re f*cking…very busy."},
            {"comedian": "shane_gillis", "sentence": "It’s like a f*cked up game of Zelda every day. You just wake up to a new quest."},
            {"comedian": "shane_gillist", "sentence": "The quest, you need to get titty behind the Home Depot"},
            {"comedian": "shane_gillis", "sentence": "Every bus is a bullet train. It’s a five-minute ride, tops."},
            {"comedian": "shane_gillis", "sentence": "I mean, she took heroin and rode roller coasters. It was… Dude, it was impossible."},
            {"comedian":'Anthony_Jeselnik', "sentence":  'Sold my passport on the street for 300 bucks to get to my prom.'},
            {"comedian":'Anthony_Jeselnik', "sentence":  "Weird joke to clap for, but sure."},
            {"comedian":'Anthony_Jeselnik', "sentence":  "My mom actually should’ve been on one of the planes that crashed on 9/11. I think."},
            {"comedian":'Anthony_Jeselnik', "sentence":  "My mom could be very racist. Very racist."},
            {"comedian":'Anthony_Jeselnik', "sentence":  "And when I did that, my mom would act weird. She’d pull me aside and say, “Anthony, who’s your new friend? Is he a drug dealer?”"},
            {"comedian":'Anthony_Jeselnik', "sentence":  "And I do not tolerate racism. That’s ignorance and I hate that."},
            {"comedian":'Anthony_Jeselnik_2', "sentence": "And I know my grandma loved it too, because it combined her two favorite things: spending time with her grandchildren, and using the ‘N’ word."},
            {"comedian":'Anthony_Jeselnik_2', "sentence": "Now I promise you… I promise you… until you’ve heard your grandfather gasp at his own wife’s funeral… …at a Methodist Church in Vicksburg, Mississippi… you are not a real comedian."},
            {"comedian":'Anthony_Jeselnik_2', "sentence": "I am a real comedian. I am a pure comedian. I think I’m one of the best comedians of all time."}, 
            {"comedian":'Ali_Wong', "sentence": "Don’t ever do that with your mom."},
            {"comedian":'Ali_Wong', "sentence": "It was like the worst experience of my life."},
            {"comedian":'Ali_Wong', "sentence": "We were screaming and fighting and yelling and it all came to a climax when she refused to let go of a Texas Instruments TI-82… manual."},
            {"comedian":'Ali_Wong', "sentence": "The manual. She don’t even know… where the calculator is."},
            {"comedian":'Ali_Wong', "sentence": "It was this calculator that bamboozled my generation. We were all required to buy it when we were in eight grade. It cost like $200. And everybody thought it was like"},
            {"comedian": 'Ali_Wong_2', "sentence": "My mom is 80, going through a full blown mid-life crisis. ‘Cause she knows that she’s got a century more to go."},
            {"comedian": 'Ali_Wong_2', "sentence": "All of her white friends, dead. Her Mexican friends, dead. Black friends, dead. I’m just kidding. She doesn’t have any black friends."},
            {"comedian": 'Ali_Wong_2', "sentence": "Old Chinese ladies, they don’t give a fuck. They got no shame. They’re like, “I’m just gonna recycle… go bald… go to the park, do this shit.”"},
            {"comedian": 'Hasan_Minhaj',"sentence": "Are you fucking kidding? Thirteen episodes for this kid? Are you kidding me?"},
            {"comedian": 'Hasan_Minhaj',"sentence": "Do you know when br0wn kids get slapped? Every br0wn birthday party."},
            {"comedian": 'Hasan_Minhaj',"sentence": "And usually it’s the kid whose birthday it is, and we stand there and point and laugh. We go, “Ah, Biju got slapped on his birthday!”"},
            {"comedian": 'Hasan_Minhaj',"sentence": "Slapping is important. It elevates your game."},
            {"comedian": 'Hasan_Minhaj',"sentence":"You ever seen an Indian kid win a spelling bee? Incredible! Ice water in the veins."},
            {"comedian": 'Hasan_Minhaj',"sentence":"You ever seen an Indian kid win a spelling bee? Incredible! Ice water in the veins."},
            {"comedian": 'Hasan_Minhaj_2',"sentence": "Getting into Stanford."},
            {"comedian": 'Hasan_Minhaj_2',"sentence": "All 30 of us? We’re all going to prom? AP Calc? Us? Me, Jehovah’s Witness girl, Korean exchange students, going to the prom? Thirty for 30? All of us?"},
            {"comedian": 'Hasan_Minhaj_2',"sentence": "Hanson, this is not funny."},
            {"comedian": 'Joe_List',"sentence": "It’s one doctor. He knows all three body parts."},
            {"comedian": 'Joe_List',"sentence": "Ear, nose, throat. “Who do ya need?” And they all kinda."},
            {"comedian": 'Joe_List',"sentence": "I thought it was called tinnitus, but then I watched a YouTube video, and the doctor in the video, he kept saying “tin-uh-dus.”"},
            {"comedian": 'Joe_List',"sentence": "This guy’s a fake ass fucking retard doctor. It’s pronounced t-ah-n-i-tus."},
            {"comedian": 'Joe_List',"sentence": "Tomato, to-mah-to. However you say it."},
            {"comedian": 'Joe_List_2',"sentence":"You feel better than everybody behind you, don’t you? You can be in the second to last row. This one guy behind, you’re like what a fucking idiot back there. Embarrassing, loser."},
            {"comedian": 'Joe_List_2',"sentence": "The whole flight he’s like. [yawns] For like five hours. [yawns] I wanted him to die, I swear to God."},
            {"comedian": 'Joe_List_2',"sentence": "First of all, you don’t need to make a noise when you yawn. That’s a decision, he’s deciding to do that."},
            {"comedian": 'Anthony_Jeselnik_3',"sentence": "Just to teach me an important lesson about brand loyalty."},
            {"comedian": 'Anthony_Jeselnik_3',"sentence": "Let her take the hit."},
            {"comedian": 'Anthony_Jeselnik_3',"sentence": "You don’t take classes. You don’t read the instructions. You just do it and figure it out."},
            {"comedian": 'Anthony_Jeselnik_3',"sentence": "Like, when I was a kid, I never got to take swimming lessons. No, my dad would pick me up and throw me in the water to teach himself CPR."},
            {"comedian": 'Ali_Wong_3',"sentence": "Which a lot of people are shocked by, because, usually, Asian-American women who, like, you know, wear these kinda glasses and have a lot of opinions, they like to date white dudes."},
            {"comedian": 'Ali_Wong_3',"sentence": "You go to any hipster neighborhood in a major city in America and that shit is turning into a Yoko Ono factory. It’s… too much."},
            {"comedian": 'Ali_Wong_3',"sentence": "I don’t know what’s wrong with these bitches"},
            {"comedian": 'Ali_Wong_3',"sentence": "You feel very… You feel very picturesque when you’re with a white dude, you know. "},
            {"comedian": 'Hasan_Minhaj_3',"sentence": "Try to understand immigrant fathers. I still can’t understand some of you. There are uncles here. None of you guys are smiling. I don’t get it. You’re going to die. Laugh. Why aren’t you laughing? You’re always stressed and always tired. You could wake up any immigrant father from a 12-hour nap, and they’d say… [angry yawn] “Why do I have to pay taxes?” You’re like, “Jesus…”"},
            {"comedian": 'Joe_List_3',"sentence": "I didn’t go to college or anything, but I know how to do therapy. I’ve been going long enough."},
            {"comedian": 'Joe_List_3',"sentence": "Whatever you’re dealing with, it’s because of your family. Look no further, than your own family."},
            {"comedian": 'Joe_List_3', "sentence": "He nailed it. These are the worst people I’ve ever met in my life. I don’t know how I didn’t see this earlier."},
            {"comedian": 'Joe_List_3', "sentence": "My biggest fear, is that when I go to the bathroom, they’ll be a man, hiding behind the shower curtain waiting"}]

context_df = pd.DataFrame(context)

In [56]:
gemma = pd.concat([no_context_df, gemma2], ignore_index=True)

In [58]:
#No Context
no_context = bipartite_metric(gemma, gt)
print("No Context Model:", no_context.select_dtypes(include='number').mean())


No Context Model: score    39.004945
dtype: float64


In [60]:
context = bipartite_metric(context_df, gt)
print("Context Model:", context.select_dtypes(include='number').mean())

Context Model: score    35.054396
dtype: float64
